# API Data Schema
Geoff Pidcock | 20190613

## Scope
Use the Python Requests library to extract information about event listings, and determine an appropriate schema.

### TODO
- Build out the query string for Sydney events
- Double check Eventbrite schema against example schema.

In [1]:
# See environment.yml for setup instructions
import sys
import os
import pandas as pd
import json
import requests

In [2]:
# Get API keys from the following places:
## Eventbrite: https://www.eventbrite.com.au/account-settings/apps
## Meetup: https://secure.meetup.com/meetup_api/key/
# This has been stored in a creds python file above the parent directory
print(os.getcwd())
os.chdir('..//..//')
print(os.getcwd())

C:\Users\Geoff Pidcock\Anaconda_Projects\Other Projects\events-aggregator\events-scraper\notebooks
C:\Users\Geoff Pidcock\Anaconda_Projects\Other Projects\events-aggregator


In [3]:
from creds import meetup_api_key, eventbrite_api_key_public
## Alternative - specify keys manually here:
# meetup_api_key = 'YOUR KEY' # https://secure.meetup.com/meetup_api/key/
# eventbrite_api_key = 'YOUR KEY' # https://www.eventbrite.com/support/articles/en_US/How_To/how-to-locate-your-eventbrite-api-user-key?lg=en_US

## Pull Data from Meetup API
[Docs](https://www.meetup.com/meetup_api/docs/)

In [4]:
# https://www.meetup.com/meetup_api/docs/find/upcoming_events/
# Todo - look into topic categories for smarter filtering - https://www.meetup.com/meetup_api/docs/find/topic_categories/
payload = dict(
    text = 'Data',
    lat = '-33.87',
    lon = '151.21',
    radius = '100',
    page = '100',
    order = 'time',
    fields = 'featured_photo,key_photo,key_photo,meta_category,group_category',
    key = meetup_api_key
)
attempt = requests.get("https://api.meetup.com/find/upcoming_events",params=payload)
data = attempt.json()
len(data['events'])

80

In [5]:
data['city']

{'id': 1000653,
 'city': 'Sydney',
 'lat': -33.87,
 'lon': 151.21,
 'state': '',
 'country': 'au',
 'zip': 'meetup1',
 'member_count': 21737}

In [6]:
i = 0
for event in data['events']:
    print("index: {} | name: {}".format(i,event['name']))
    i += 1

index: 0 | name: TWGAS #6
index: 1 | name: The Humans of Ai Podcast
index: 2 | name: GA & Data Science Breakfast Meetup presents: The Rise of Automation
index: 3 | name: Learn to code at your own pace
index: 4 | name: SCP: Facebook Hacker Cup Special
index: 5 | name: Advanced-level volleyball
index: 6 | name: Sydney: Home grown IoT. Presented by Aaron Powell
index: 7 | name: Alteryx User Group Sydney
index: 8 | name: Docker Meetup - RSVP
index: 9 | name: Newy DevOps & Cloud (Infracoders) June 2019 Meetup
index: 10 | name: Unleash the power of data fabrics and unlearn old data management practices...
index: 11 | name: Kick off Meetup
index: 12 | name: Python Basics Class 
index: 13 | name: #ShutUpandCode
index: 14 | name: OpenMined Sydney - Starter Session
index: 15 | name: 匠人线上公开课 | 求职新手训练营 , 嘉宾来自 Facebook, Canva, ANZ, Rokt, CoreLogic等
index: 16 | name: Content Marketing Deepdive: Connect and Convert
index: 17 | name: June DevOps Meetup
index: 18 | name: Why The Gender Pay Gap in Tech 

In [7]:
data['events'][52]
# 48, 52

{'created': 1558583944000,
 'duration': 5400000,
 'id': '261696147',
 'name': 'LoRa & The Things Network (TTN) Workshop',
 'rsvp_limit': 8,
 'date_in_series_pattern': False,
 'status': 'upcoming',
 'time': 1562227200000,
 'local_date': '2019-07-04',
 'local_time': '18:00',
 'updated': 1558583944000,
 'utc_offset': 36000000,
 'waitlist_count': 0,
 'yes_rsvp_count': 4,
 'venue': {'id': 24974651,
  'name': 'Core Electronics',
  'lat': -32.94047164916992,
  'lon': 151.71743774414062,
  'repinned': True,
  'address_1': 'Unit 18, 132 Garden Grove Parade',
  'city': 'Newcastle',
  'country': 'au',
  'localized_country_name': 'Australia'},
 'group': {'created': 1483689782000,
  'name': 'Core Electronics Workshops',
  'id': 21785246,
  'join_mode': 'open',
  'lat': -32.91999816894531,
  'lon': 151.75,
  'urlname': 'Core-Electronics-Workshops',
  'who': 'Makers',
  'localized_location': 'Newcastle, Australia',
  'state': '',
  'country': 'au',
  'region': 'en_US',
  'timezone': 'Australia/Sydney

## Meetup Findings
- Not sure if list is comprehensive - might make sense to assemble a list of data related groups, and then iterate through each 
- Will need to process datetimes from epoc to calendar before writing to database

In [8]:
# DB Schema - Postgres 9.3+
sql = """
CREATE TABLE IF NOT EXISTS raw_events(
    event_id SERIAL PRIMARY KEY,
    source_platform_id CHAR VARYING(20)
    source_platform CHAR VARYING(50),
    event_city CHAR VARYING(50),
    event_date_local DATE,
    event_time_local TIME,
    event_name TEXT,
    event_organiser TEXT,
    event_location_name TEXT,
    event_location_address TEXT,
    event_lat NUMERIC(20,16),
    event_lon NUMERIC(20,16),
    registration_link TEXT,
    description TEXT,
    event_or_group_photo Text,
    event_category CHAR VARYING(100),
    event_category CHAR VARYING(100)
)
"""

In [9]:
# Response Schema based on Meetup
[
    {
        'event_id': '1',
        'source_platform_id': '261218488',
        'source_platform': 'Meetup',
        'event_city': 'Sydney',
        'event_date': '2019-06-14',
        'event_time': '07:30',
        'event_name': 'GA & Data Science Breakfast Meetup presents: The Rise of Automation',
        'event_organiser': 'Data Science Breakfast Meetup',
        'event_location_name': 'GA Sydney (Main Campus)',
        'event_location_address': 'The Podium Building, 1 Market St Sydney AU',
        'event_lat': '-33.869998931884766',
        'event_lon': '151.20460510253906',
        'registration_link': 'https://www.meetup.com/The-Sydney-Data-Science-Breakfast-Meetup-Group/events/261218488/',
        'description': '<p>Note: this is a partnered event with General Assembly. Please make sure to register using the General Assembly event page, linked here: <a href="https://ga.co/2PPoZWd" class="linkified">https://ga.co/2PPoZWd</a></p> <p>****<br/>Abstract:<br/>There has been a lot of attention in the media surrounding the rise of automation. As advanced technologies such as robotic process automation, machine learning, and artificial intelligence have matured, companies have found practical applications to these new technologies.</p> <p>As part of this evolution, Intelligent Automation has become a topic of interest for business leaders across industries looking to combine cognitive capabilities with robotic process technologies to create a "living" system that can go beyond mundane and repeatable tasks. This is extremely advantageous to any organization that can implement these systems seamlessly but there is still a delta between the ideation of AI integration. and the ability to put these plans into effect. This delta can instill fear and anxiety around using AI as well as the added question of the ethical implications of AI such as Facial Recognition and targeted advertising.</p> <p>Join General Assembly and the Data Science Breakfast Meetup as we present a panel of experts at the forefront of AI incorporation for an engaging conversation that will touch on.</p> <p>*****<br/>Agenda:<br/>07:30 - arrival and networking<br/>07:50 - panel kick off<br/>08:30 - panel Q&amp;A<br/>09:00 - more networking (and anyone who needs to head off can leave)<br/>09:30 - close</p> <p>*****<br/>Speaker BIO\'s:</p> <p>Anthony Tockar (Moderator):<br/>Anthony Tockar is director and cofounder at Verge Labs, a new type of AI company focused on the applied side of machine learning. A jack-of-all-trades, he has worked on problems across insurance, technology, telecommunications, loyalty, sports betting and even neuroscience. He qualified as an actuary, then moved into data science, completing an MS in Analytics at the prestigious Northwestern University.</p> <p>After hitting the headlines with his posts on data privacy at Neustar, he returned to Sydney to practice as a data scientist and cofounded the Minerva Collective, a not-for-profit focused on using data for social good, as well as multiple meetup groups. His key missions are to extend the reach and impact of data science to help people, and to assist Australian businesses to become more data driven.</p> <p>Sam Zheng (Panelist):<br/>Sam is Co-founder/CEO of Curious Thing - a voice-based AI interviewer for talent acquisition startup. Sam is a tech entrepreneur, self-taught engineer, and qualified actuary. Before Curious Thing, Sam was Co-founder/CTO of Hyper Anna, an AI for business analytics startup.</p> <p>Dima Galat (Panelist):<br/>Dima learned to program in Assembly on an i486, back when disk sizes were measured in megabytes. He always saw programming as a tool for facilitating communication between disparate data sources and end users.</p> <p>After his first encounter with data mining a decade ago, he became obsessed with applied machine learning, which supercharges this communication process. He has a background in computer vision productisation, data engineering, and a variety of analytics projects for clients ranging from financial institutions to United Nations.</p> <p>Usman Shahbaz (panelist):<br/>Usman is an experienced leader with more than 14 years of rich experience in applying product, network, risk-assurance and consumer analytics to drive actionable business outcomes. His core specialties include Advanced Analytics, Machine &amp; Deep Learning, Statistical Modelling and Optimisation. Usman is currently enrolled for a PhD in Machine Learning. He also holds an MBA and a Bachelor’s degree in Electrical Engineering.</p> <p>Passiona Cottee (panelist):<br/>Data Scientist, Commonwealth Bank, Co-lecturer at UTS</p> ',
        'event_or_group_photo': 'https://secure.meetupstatic.com/photos/event/c/6/6/5/highres_481070789.jpeg',
        'event_format': 'panel',
        'event_category': 'automation'       
    }
]

[{'event_id': '1',
  'source_platform_id': '261218488',
  'source_platform': 'Meetup',
  'event_city': 'Sydney',
  'event_date': '2019-06-14',
  'event_time': '07:30',
  'event_name': 'GA & Data Science Breakfast Meetup presents: The Rise of Automation',
  'event_organiser': 'Data Science Breakfast Meetup',
  'event_location_name': 'GA Sydney (Main Campus)',
  'event_location_address': 'The Podium Building, 1 Market St Sydney AU',
  'event_lat': '-33.869998931884766',
  'event_lon': '151.20460510253906',
  'registration_link': 'https://www.meetup.com/The-Sydney-Data-Science-Breakfast-Meetup-Group/events/261218488/',
  'description': '<p>Note: this is a partnered event with General Assembly. Please make sure to register using the General Assembly event page, linked here: <a href="https://ga.co/2PPoZWd" class="linkified">https://ga.co/2PPoZWd</a></p> <p>****<br/>Abstract:<br/>There has been a lot of attention in the media surrounding the rise of automation. As advanced technologies such 

## Pull Data from Eventbrite
Involves setting up an app and getting the app's api public key.<br>
*References:*
- [EventBrite API Docs](https://www.eventbrite.com/platform/api)

In [10]:
url = "https://www.eventbriteapi.com/v3/events/search/"

querystring = {"token":eventbrite_api_key_public}

payload = ""

headers = {
   'cache-control': "no-cache",
   }

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

response.status_code

200

In [11]:
data = response.json()

In [12]:
print(len(data['events']))
# Pagination will likely be needed

50


In [13]:
print(data['events'][0])
# might make sense to pretty print this, and sort out schema

{'name': {'text': 'Big Tigger Hosts Concert Weekend Kickoff at Suite Lounge - RSVP NOW', 'html': 'Big Tigger Hosts Concert Weekend Kickoff at Suite Lounge - RSVP NOW'}, 'description': {'text': '\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\xa0\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nSUITE LIFE FRIDAYS\r\nHOSTED BY\r\nBIG TIGGER\r\nFREE TIL 12AM W/ RSVP\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\xa0\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n©2019 Furious | Peachtree Road, Atlanta Ga\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n', 'html': '<TABLE ID="container" CLASS="container logoless" STYLE="color: #000000; font-family: Times; table-layout: fixed; border-collapse: collapse; border-spacing: 0px; width: 640px;">\r\n<TBODY>\r\n<TR>\r\n<TD ID="" STYLE="padding-top: 20px;">\r\n<TABLE ID="outer_wrapper" STYLE="table-layout: fixed; border-colla